In [1]:
!pip install -q -U google-generativeai

In [2]:
import unicodedata

In [4]:
from google.colab import userdata
import numpy as np
import pandas as pd
import random
import google.generativeai as genai

# Configurações da API do Google AI
GOOGLE_API_KEY = userdata.get('token')
genai.configure(api_key=GOOGLE_API_KEY)

# Função para normalizar texto
def normalizar_texto(texto):
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    texto = texto.lower().strip()
    return texto

# Selecionar o modelo de embedding
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    model = m.name
    break

# Função para gerar embeddings de texto
def embed_fn(nome, endereco, tipo_de_doacao):
    descricao = f"{endereco} - {tipo_de_doacao}"
    return genai.embed_content(
        model=model,
        content=descricao,
        title=nome,
        task_type="RETRIEVAL_DOCUMENT"
    )["embedding"]


# Carregar dados de doações de um arquivo CSV
df = pd.read_csv("/content/doacoes.csv", encoding="utf-8")
df["Cidade"] = df["Cidade"].str.strip()

# Gerar embeddings para cada local de doação
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Nome"], row["Endereco"], row["Tipo_de_Doacao"]), axis=1)

# Função para buscar locais de doação com base na consulta do usuário
def gerar_e_buscar_consulta(consulta, base, model, num_resultados=3):
  # Extrair a cidade da consulta do usuário
  palavras_consulta = [normalizar_texto(palavra) for palavra in consulta.split()]
  cidades_unicas = set(normalizar_texto(cidade) for cidade in base["Cidade"].unique())
  cidade_consulta = None
  for cidade in cidades_unicas:
    if cidade in palavras_consulta:
      cidade_consulta = cidade
      break

  # Filtrar o DataFrame pela cidade, se uma cidade for encontrada na consulta
  if cidade_consulta:
        base = base[base["Cidade"].apply(normalizar_texto) == cidade_consulta]

  # Gerar um embedding da consulta do usuário
  embedding_da_consulta = genai.embed_content(
      model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  # Calcular a similaridade (produto escalar) entre a consulta e cada local
  produtos_escalares = np.dot(np.stack(base["Embeddings"]), embedding_da_consulta)

  # Encontrar os índices dos N locais mais similares
  indices_ordenados = np.argsort(produtos_escalares)[-num_resultados:][::-1]  # Inverter a ordem para decrescente

  # Randomizar a ordem dos índices para apresentar resultados variados
  random.shuffle(indices_ordenados)

  # Retornar os N locais mais similares à consulta
  return base.iloc[indices_ordenados]

# Exemplo de consulta
consulta = "Onde posso doar brinquedos em Belo Horizonte ?"
resultado = gerar_e_buscar_consulta(consulta, df, model)
print(resultado)

# Gerar uma resposta textual usando o modelo de linguagem Gemini
nome_local = resultado["Nome"]
cidade_local = resultado["Cidade"]
endereco_local = resultado["Endereco"]
horario_local = resultado["Horario"]
prompt = f"Responda à pergunta '{consulta}' usando as seguintes informações: Nome do local: {nome_local}, Cidade: {cidade_local} Endereço: {endereco_local}, Horário: {horario_local}"

generation_config = {"candidate_count": 1, "temperature": 0.5}
model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)



           Cidade                             Nome                 Endereco  \
43  Caxias do Sul   Patronato Santa Rita de Cássia    Rua Madre Teresa, 456   
20   Campo Grande  Centro Espírita Luz da Caridade  Rua dos Girassóis, 1011   
19         Maceió              Abrigo SOS Crianças   Rua dos Coqueiros, 789   

                                       Tipo_de_Doacao  \
43  Roupas em bom estado para crianças e adolescen...   
20  Alimentos não perecíveis, roupas em bom estado...   
19  Brinquedos, livros, materiais escolares, roupa...   

                           Horario  \
43  Segunda a Sexta, das 8h às 14h   
20  Terça, Quinta e Sábado, 9h-13h   
19         Terça e Quinta, 14h-17h   

                                           Embeddings  
43  [0.043702725, -0.0390561, -0.04862553, -0.0027...  
20  [0.022739999, -0.024239067, -0.07516284, -0.01...  
19  [0.012811212, -0.0069083795, -0.05625487, -0.0...  
Centro Espírita Luz da Caridade


In [5]:
import ipywidgets as widgets
from IPython.display import HTML, display
# ... (seu código de API, normalização, embeddings, etc.) ...

# Estilo CSS para centralizar os elementos e formatar o texto
style = """
.container {
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    height: 150px; /* Ajuste a altura conforme necessário */
}
.output {
    white-space: pre-wrap;
    margin-bottom: 10px;
    text-align: left;
}
.output strong {
    font-weight: bold;
}
.mensagem-recebida {
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
}
"""
# Título
titulo = widgets.HTML(value="<h1>Sistema de Busca de Doações</h1>")
# Widget de entrada de texto
message_input = widgets.Text(
    placeholder='Digite sua pergunta (ex: Onde posso doar roupas em São Paulo?)',
    description='Pergunta:'
)
# Botão de busca
button = widgets.Button(description="Buscar")

# Widget de saída para exibir os resultados
output = widgets.Output()

# Widget de entrada de texto
message_input = widgets.Text(
    value='',
    placeholder='Digite sua pergunta (ex: Onde posso doar roupas em São Paulo?)',
    description='Pergunta:',
    disabled=False,
    layout=widgets.Layout(width='500px'))

# Função para lidar com a mensagem e realizar a busca
def handle_message(sender):
    consulta = message_input.value
    with output:
        display(HTML(f"<div class='mensagem-recebida'><strong>Mensagem recebida:</strong> {consulta}</div>"))
        resultados = gerar_e_buscar_consulta(consulta, df, model, num_resultados=3)  # Obter 3 resultados

        # Criar lista numerada HTML
        lista_html = "<ol>"
        for i in range(len(resultados)):
            resultado = resultados.iloc[i]
            nome_local = resultado["Nome"]
            cidade_local = resultado["Cidade"]
            endereco_local = resultado["Endereco"]
            horario_local = resultado["Horario"]
            prompt = f"Responda à pergunta '{consulta}' usando as seguintes informações: Nome do local: {nome_local}, Cidade: {cidade_local} Endereço: {endereco_local}, Horário: {horario_local}"
            response = model_2.generate_content(prompt)
            lista_html += f"<li>{response.text}</li><br>"
        lista_html += "</ol>"

        # Renderizar a saída como HTML
        output_html = f"<div class='output'>{lista_html}</div>"
        display(HTML(output_html))

# Função para limpar a lista de resultados
def clear_output(sender):
    output.clear_output()

# Botão para disparar a busca
button = widgets.Button(description="Buscar")
button.on_click(handle_message)

# Botão para limpar a saída
clear_button = widgets.Button(description="Limpar")
clear_button.on_click(clear_output)

# Conectar o botão à função
button.on_click(handle_message)

# Label para dados fictícios
dados_ficticios_label = widgets.HTML(value="Dados fictícios para fins de demonstração.*", layout=widgets.Layout(margin="10px 0"))

# Organizar widgets na caixa horizontal
box = widgets.HBox([message_input, button, clear_button])


# Organizar os elementos na caixa vertical e centralizar
container = widgets.VBox([titulo, box, dados_ficticios_label, output], layout=widgets.Layout(align_items='center'))

# Exibir o estilo CSS e o container
display(HTML(style))
display(container)